In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys, os, gc, types
import time

In [9]:
import os

root_paths = [
    "/data/kaggle-instacart",
    "/Users/jiayou/Dropbox/珺珺的程序/Kaggle/Instacart",
    "/Users/jiayou/Dropbox/Documents/珺珺的程序/Kaggle/Instacart"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
path = root
sbpath = os.path.join(root, 'sb')

In [10]:
class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " starts...")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print('{} done: {:.2f}s'.format(self.process_name, end_time - self.begin_time))

In [11]:
import gc
import pandas as pd
import numpy as np
import os
import json
import sklearn.metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from scipy.sparse import dok_matrix, coo_matrix
from sklearn.utils.multiclass import  type_of_target

In [2]:
gorders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id': np.uint32,
                                                              'user_id': np.uint32,
                                                              'eval_set': 'category',
                                                              'order_number': np.uint8,
                                                              'order_dow': np.uint8,
                                                              'order_hour_of_day': np.uint8
                                                              })

gorder_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order': np.uint8,
                                                                                  'reordered': bool})
gorder_prior = gorder_prior.merge(gorders[['order_id', 'user_id']], on='order_id', how='left')

gorder_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order': np.uint8,
                                                                                  'reordered': bool})
gorder_train = gorder_train.merge(gorders[['order_id', 'user_id']], on='order_id', how='left')

aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle':'category'})
departments = pd.read_csv(
    os.path.join(path, "departments.csv"), 
    dtype={'department_id':np.uint8, 'department': 'category'})
products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                  'aisle_id': np.uint8,
                                                                  'department_id': np.uint8})

In [3]:
def aug_name(s, ms):
    return 'aug{}-{}'.format(s, ms)

def orig_name(shard):
    return 'orig{}'.format(shard)

def load(shard=None, nshards=None, aug=None):
    if aug is None:
        global gorders, gorder_prior, gorder_train
        orders = gorders.loc[gorders.user_id % nshards == shard, :]
        
        order_prior = gorder_prior.loc[gorder_prior.user_id % nshards == shard, :]
        order_prior.drop('user_id', inplace = True, axis=1)
        
        order_train = gorder_train.loc[gorder_train.user_id % nshards == shard, :]
        order_train.drop('user_id', inplace = True, axis=1)
    else:
        pf = os.path.join(root, 'aug', 'order_products__prior.{}.csv'.format(aug))
        tf = os.path.join(root, 'aug', 'order_products__train.{}.csv'.format(aug))
        of = os.path.join(root, 'aug', 'orders.{}.csv'.format(aug))
        orders = pd.read_csv(of, dtype={
            'order_id': np.uint32,
            'user_id': np.uint32,
            'eval_set': 'category',
            'order_number': np.uint8,
            'order_dow': np.uint8,
            'order_hour_of_day': np.uint8
        })
        order_prior = pd.read_csv(pf, dtype={
            'order_id': np.uint32,
            'product_id': np.uint16,
            'add_to_cart_order': np.uint8,
            'reordered': bool
        })
        order_train = pd.read_csv(tf, dtype={
            'order_id': np.uint32,
            'product_id': np.uint16,
            'add_to_cart_order': np.uint8,
            'reordered': bool
        })
    return (orders, order_prior, order_train)

In [8]:
def process_shard(s):
    aug = s[0]
    if aug:
        orders, order_prior, order_train = load(aug=aug_name(s[1], s[2]))
        name = aug_name(s[1], s[2])
    else:
        orders, order_prior, order_train = load(shard=s[1], nshards=s[2])
        name = orig_name(s[1])
        
    global aisles, departments, products

########## user's product list

# def create_products():

#     print('loaded')

    torders = orders.loc[orders.eval_set == 'prior', :]
    orders_user = torders[['order_id', 'user_id']]
    previous_products = pd.merge(order_prior, orders_user, on='order_id')
    previous_products = previous_products.loc[:, ['user_id', 'product_id']].drop_duplicates()

#     print('save')
#     print(labels.shape)
#     print(labels.columns)
#     labels.to_pickle('sb/previous_products.pkl')
#     return labels

# labels = create_products()



########## abt skeleton without features

# def split_data_set():
    labels = previous_products.copy()
    torders = orders.loc[(orders.eval_set == 'train') | (orders.eval_set == 'test'), :]
    labels = pd.merge(labels, torders[['order_id', 'user_id', 'eval_set']], on='user_id').drop(['user_id'], axis=1)

#     print('data is loaded')

    torders = np.unique(labels.order_id)

    size = torders.shape[0] // folds


    current = torders

    current = labels.loc[np.in1d(labels.order_id, current), :]

    current = pd.merge(
        order_train.drop(['add_to_cart_order'], axis=1), 
        current, on=['order_id', 'product_id'], how='right')
    current.reordered.fillna(False, inplace=True)
    chunk = current

# folds = split_data_set()



########## intervals a user purchases a product

# def orders_comsum():
#     path = root
#     sbpath = os.path.join(root, 'sb')

    aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle': 'category'})
    departments = pd.read_csv(os.path.join(path, "departments.csv"),
                              dtype={'department_id': np.uint8, 'department': 'category'})
    orders, order_prior, order_train = load()

    products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                      'aisle_id': np.uint8,
                                                                      'department_id': np.uint8})

    labels = pd.read_pickle(os.path.join(sbpath, 'chunk_0.pkl'))
    user_product = previous_products.copy()

    order_comsum = orders[['user_id', 'order_number', 'days_since_prior_order']].groupby(['user_id', 'order_number'])\
            ['days_since_prior_order'].sum().groupby(level=[0]).cumsum().reset_index().rename(columns={'days_since_prior_order':'days_since_prior_order_comsum'})

    # order_comsum['days_since_prior_order_comsum'].fillna(0, inplace=True)
    order_comsum.to_pickle('sb/orders_comsum.pkl')

    order_comsum2 = pd.merge(order_comsum, orders, on=['user_id', 'order_number'])[['user_id', 'order_number', 'days_since_prior_order_comsum', 'order_id']]

    order_product = pd.merge(order_prior, orders, on='order_id')[['order_id', 'product_id', 'eval_set']]
    order_product_train_test = labels[['order_id', 'product_id', 'eval_set']]

    order_product = pd.concat([order_product, order_product_train_test])

    order_product = pd.merge(order_product, order_comsum2, on='order_id')

    print(order_product.columns)

    order_product = pd.merge(order_product, user_product, on=['user_id', 'product_id'])

    temp = order_product.groupby(['user_id', 'product_id', 'order_number'])['days_since_prior_order_comsum'].sum().groupby(level=[0, 1]).apply(lambda x: np.diff(np.nan_to_num(x)))
    temp = temp.to_frame('periods').reset_index()

    temp.to_pickle('sb/product_period.pkl')

    aggregated = temp.copy()
    aggregated['last'] = aggregated.periods.apply(lambda x: x[-1])
    aggregated['prev1'] = aggregated.periods.apply(lambda x: x[-2] if len(x) > 1 else np.nan)
    aggregated['prev2'] = aggregated.periods.apply(lambda x: x[-3] if len(x) > 2 else np.nan)
    aggregated['median'] = aggregated.periods.apply(lambda x: np.median(x[:-1]))
    aggregated['mean'] = aggregated.periods.apply(lambda x: np.mean(x[:-1]))
    aggregated.drop('periods', axis=1, inplace=True)

    aggregated.to_pickle('sb/product_periods_stat.pkl')
    
    return (order_comsum, temp, aggregated)

oc = orders_comsum()



########## user_dep_stat and user_aisle_stat



def user_product_rank():
    path = root
    sbpath = os.path.join(root, 'sb')

    aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle': 'category'})
    departments = pd.read_csv(os.path.join(path, "departments.csv"),
                              dtype={'department_id': np.uint8, 'department': 'category'})

    orders, order_prior, order_train = load()

    products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                      'aisle_id': np.uint8,
                                                                      'department_id': np.uint8})

    order_train = pd.read_pickle(os.path.join(sbpath, 'chunk_0.pkl'))

    
    orders_products = pd.merge(orders, order_prior, on="order_id")

    orders_products_products = pd.merge(orders_products, products[['product_id', 'department_id', 'aisle_id']],
                                        on='product_id')

    user_dep_stat = orders_products_products.groupby(['user_id', 'department_id']).agg(
        {'product_id': lambda x: x.nunique(),
         'reordered': 'sum'
         })
    user_dep_stat.rename(columns={'product_id': 'dep_products',
                                  'reordered': 'dep_reordered'}, inplace=True)
    user_dep_stat.reset_index(inplace=True)
    print(user_dep_stat.columns)
    user_dep_stat.to_pickle('sb/user_department_products.pkl')

    user_aisle_stat = orders_products_products.groupby(['user_id', 'aisle_id']).agg(
        {'product_id': lambda x: x.nunique(),
         'reordered': 'sum'
         })
    user_aisle_stat.rename(columns={'product_id': 'aisle_products',
                                    'reordered': 'aisle_reordered'}, inplace=True)
    user_aisle_stat.reset_index(inplace=True)
    print(user_aisle_stat.columns)
    user_aisle_stat.to_pickle('sb/user_aisle_products.pkl')
    
    return (user_dep_stat, user_aisle_stat)

upr = user_product_rank()

In [ ]:
def process_shards(shards):
    for s in shards:
        with tick_tock("Process shard {}".format(s)):
            process_shard(shard=s)
        gc.collect()

In [ ]:
from multiprocessing import Process

n_shards = 32

shards = [(False, s, 64) for s in range(64)] + [(True, s, ms) for ms in range(2) for s in range(32)]

jobs = []
for s in range(n_shards):
    cur_shards = [shards[i] for i in range(len(shards)) if i%n_shards == s]
    p = Process(target=process_shards, args=(cur_shards, down_sample))
    p.start()
    jobs.append(p)
    
for p in jobs:
    p.join()

print("\n\nAll done.")